## Image Captioning 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import requests
from PIL import Image
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data.csv')

In [ ]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large", torch_dtype=torch.float16).to("cuda")
# Initialize Blip processor and model

# Function to generate caption for an image
def generate_caption(image_path):
    # Load image
    raw_image = Image.open(image_path).convert('RGB')

    # Perform conditional image captioning
    text = "a photograph of"
    inputs = processor(raw_image, text, return_tensors="pt").to("cuda", torch.float16)
    out = model.generate(**inputs, min_length=20)  # Adjust min_length as needed
    caption = processor.decode(out[0], skip_special_tokens=True)

    return caption
# Function to process each row

def process_row(row):
    # Extract URLs from 'originalPhotos' column
    photo_urls = eval(row['originalPhotos'])  # Convert string representation to list

    # Initialize a list to store captions
    captions = []
    try:
        # Loop through each photo URL
        for url in photo_urls:
            # Send request to download the image
            response = requests.get(url)
            image_path = url.split('/')[-1]  # Extract image filename
            with open(image_path, 'wb') as f:
                f.write(response.content)

            # Generate caption for the image
            caption = generate_caption(image_path)
            captions.append(caption)

        # Return concatenated captions
        return '; '.join(captions)
    except:
      print("Error")
# Function to process and save data every specified number of rows
def process_and_save_from_index(df, start_index, batch_size, output_file_prefix='/content/drive/MyDrive/processed_data_'):
    total_rows = len(df)
    num_batches = (total_rows - start_index) // batch_size + 1

    for batch_index in range(num_batches):
        current_index = start_index + batch_index * batch_size
        end_index = min(current_index + batch_size, total_rows)

        if current_index >= total_rows:
            break

        batch_df = df.iloc[current_index:end_index]
        batch_df['image_captions'] = batch_df.apply(process_row, axis=1)

        output_file = f"{output_file_prefix}{current_index + 1}_to_{end_index}.csv"
        batch_df.to_csv(output_file, index=False)

        print(f"Processed and saved rows {current_index + 1} to {end_index} to {output_file}")
        # break

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
start_index = 501# Change this according to your requirement
batch_size = 100

# Process and save the data
process_and_save_from_index(df, start_index, batch_size)

print("All data processed and saved.")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
<ipython-input-4-deb3982ce8ab>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_df['image_captions'] = batch_df.apply(process_row, axis=1)


Processed and saved rows 302 to 401 to /content/drive/MyDrive/processed_data_302_to_401.csv


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Error


<ipython-input-4-deb3982ce8ab>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_df['image_captions'] = batch_df.apply(process_row, axis=1)


Processed and saved rows 402 to 501 to /content/drive/MyDrive/processed_data_402_to_501.csv


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
